In [ ]:
%use plotly
import kotlinx.coroutines.*
import java.time.Duration
import java.time.Instant

val data = runBlocking { readAllUData("./data/log.anon") + readAllUData("./data/log2.anon") }

In [ ]:
enum class SigningAlgorithm {
    RSA, ECDSA_P_256, ECDSA_P_384, Ed25519, Ed448
}

fun plot(udata: List<UData>, title: String): Plot {
    val splitByAlgo = udata.groupBy { ud ->
        when (ud.algo.toInt()) {
            5 -> SigningAlgorithm.RSA
            7 -> SigningAlgorithm.RSA
            8 -> SigningAlgorithm.RSA
            10 -> SigningAlgorithm.RSA
            13 -> SigningAlgorithm.ECDSA_P_256
            14 -> SigningAlgorithm.ECDSA_P_384
            15 -> SigningAlgorithm.Ed25519
            16 -> SigningAlgorithm.Ed448
            else -> throw NotImplementedError()
        }
    }.toList()
    val total = udata.size
    val percentages = splitByAlgo.map { (_, udl) -> udl.size }
    val alogrithm = splitByAlgo.map { (alg, udl) ->
        val percentage = udl.size.toDouble() / total * 100
        val progressString = "%.2f".format(percentage)
        val algName = when (alg) {
            SigningAlgorithm.ECDSA_P_256 -> "ECDSA P-256"
            SigningAlgorithm.ECDSA_P_384 -> "ECDSA P-384"
            else -> alg.toString()
        }
        "$algName: $progressString%"
    }

    return Plotly.plot {
        pie {
            values(percentages)
            labels(alogrithm)
            textinfo = TextInfo.`label`
        }
        layout {
            width = 500
            height = 375
            title {
                text = title
            }
        }
    }
}

In [ ]:
plot(data, "Percentage of validations per signing algorithm")

In [ ]:
plot(
    data.filter { u -> u.domainName.size <= 2 && u.sub == false },
    "Percentage of validations per algorithm for root zone and TLDs"
)

In [ ]:
plot(
    data.filter { u -> u.domainName.size <= 2 && u.sub == false }.distinctBy { u -> Pair(u.domainName, u.algo) },
    "Percentage of validations per algorithm for root zone and TLDs without duplicate algorithm and subdomain"
)

In [ ]:
plot(
    data.filter { u -> u.sub == true },
    "Percentage of validations per algorithm for subdomains of prefixes"
)